In [2]:
from model.hierarchical_self_attention_model import HSA_model, HSA_VAE
import tensorflow as tf
import yaml

In [3]:
%load_ext autoreload
%autoreload 2

In [25]:
hparam_file = open('configs/hyperparameters.yaml', mode='r')
hyperparameters = yaml.load(hparam_file, Loader=yaml.FullLoader)

In [26]:
hyperparameters

{'HSA_model': {'modality_indices': {'opp': [0, 9, 18, 27, 36, 45, 61, 77],
   'mex': [0, 3, 6]},
  'dff': 512,
  'd_model': 64,
  'num_heads': 4,
  'dropout': 0.2},
 'train': {'batch_size': 64,
  'epochs': 100,
  'val_split': 0.1,
  'learning_rate': 0.001}}

In [27]:
n_window, n_timesteps, n_features, n_outputs = 36, 25, 77, 6

hsa_model = HSA_model(hyperparameters['HSA_model']['modality_indices']['opp'], n_window, n_timesteps, n_features, n_outputs, d_model=hyperparameters['HSA_model']['d_model'], num_heads = hyperparameters['HSA_model']['num_heads'], dff=hyperparameters['HSA_model']['dff'], dropout_rate=hyperparameters['HSA_model']['dropout'])

model = hsa_model.get_model()

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=hyperparameters['train']['learning_rate']), metrics='accuracy')
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 36, 25, 77)]      0         
_________________________________________________________________
multi_window_encoder_1 (Mult ((None, None, 64), (None, 1249472   
_________________________________________________________________
modality_encoder_block_15 (M (None, None, 64)          170048    
_________________________________________________________________
combined_sensor_self_attenti ((None, 64), (None, 1, No 82880     
_________________________________________________________________
dense_232 (Dense)            (None, 6)                 390       
Total params: 1,502,790
Trainable params: 1,502,790
Non-trainable params: 0
_________________________________________________________________


In [23]:
hsa_vae_model = HSA_VAE(base_model=model, feature_dim=hyperparameters['HSA_model']['d_model']).get_model()

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters['train']['learning_rate'])
hsa_vae_model.compile(optimizer)    